In [56]:
# https://python.langchain.com/docs/tutorials/extraction/
# Tools: https://python.langchain.com/docs/concepts/tool_calling/
# https://platform.openai.com/docs/guides/function-calling

In [77]:
from typing import Optional
from pydantic import BaseModel, Field, field_validator
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama
import os

In [78]:
class Person(BaseModel):
    """Information about a person."""
    name: Optional[str]             = Field(default=None, description="The name of the person")
    hair_color: Optional[str]       = Field(default=None, description="The color of the person's hair if known")
    height_in_meters: Optional[str] = Field(default=None, description="Height measured in meters")
    
    @field_validator("height_in_meters", mode="before")
    def convert_height_to_string(cls, value):
        if isinstance(value, (float, int)):
            return str(value)
        return value    

In [79]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
        You are an expert extraction algorithm.
        Extract the following fields about a person from the given passage.

        Ensure the following:
        - `name` must always be a string.
        - `hair_color` must always be a string.
        - `height_in_meters` must always be a string, even if it represents a numerical value.

        Return all fields in the correct format based on the schema.

        Passage:
        {text}
    """),
    ("human", "")
])

In [80]:
llm = ChatOllama(base_url=os.getenv("OLLAMA_SERVER"),model = "llama3.2:latest",temperature = 0.8,num_predict = 256,)

In [81]:
structured_llm = llm.with_structured_output(schema=Person)

In [82]:
text   = "Alan Smith is 10 feet tall and has red hair."
prompt = prompt_template.invoke({"text": text})
response = structured_llm.invoke(prompt)
print(response)

name='Alan Smith' hair_color='red' height_in_meters='10 feet'
